<a name="top"></a><img src="source/SpinalHDL.png" alt="SpinalHDL based on Scala" style="width:320px;" />

运行SpinalHDL代码之前，务必加载SpinalHDL的库 
（**注**：*第一次运行时需要下载库，可能会有点慢，请耐心等待*）

## Chapter


In [ ]:
val path = System.getProperty("user.dir") + "/source/load-spinal.sc"
interp.load.module(ammonite.ops.Path(java.nio.file.FileSystems.getDefault().getPath(path)))

In [ ]:
class Person3 {}
class Student3 extends Person3

List作为参数传入

In [ ]:
Array(Range(1,10):_*)

Array.fill(8)(Random.nextInt%256)

In [ ]:
val p: Person3 = new Student3
val p0: Person3 = new Person3
val s: Student3 = p.asInstanceOf[Student3]

## 文件操作
- 读文件
```scala
scala.io.Source.fromFile(sp.generatedSourcesPaths.head).mkString     
```
- 写的三种方法
  + java io
```scala
import java.io.PrintWriter
new PrintWriter("tmp/fixpoint.txt"){write(strings.foldLeft("")(_+_+"\n"));close}
```
  + java io 
```scala
import java.io.{PrintWriter，File}
val fp = new PrintWriter(new File("tmp/fixpoint.txt"))
fp.write(strings.foldLeft("")(_+_+"\n"))
fp.close()  
```
  + scala reflect(2.12)
```scala
reflect.io.File("tmp/fixpoint.txt").writeAll(strings.foldLeft("")(_+_+"\n"))
``` 

In [ ]:
val a = new Area

In [ ]:
class Counter0(start: BigInt, end: BigInt) extends ImplicitArea[UInt] {    
    val willIncrement: Bool = False.allowOverride
    val willClear: Bool = False.allowOverride
    
    def clear() = willClear := True
    def increment() = willIncrement  := True
    
    val valueNext =UInt(log2Up(end + 1) bit)    
    val value = RegNext(valueNext) init(start)
    
    val willOverflowIfInc: Bool = value === end 
    val willOverflow:Bool = willOverflowIfInc && willIncrement
    
    when (willOverflow){
        valueNext := U(start)        
    }.elsewhen(willClear) {
        valueNext := start + 1
    }.elsewhen(willIncrement) {
        valueNext := value + U(1)
    }.otherwise{
        valueNext := value  
    }
   override def implicitValue: UInt = this.value
}

In [ ]:
class T1 extends Component{
    val o = out UInt()
    val ready,clc = in Bool()
    val o2 = out Bool()
    val u23 = new Counter0(12,31)
    when(ready){u23.increment}
    when(clc){u23.clear}
    o := u23
    o2 := u23.willOverflow
    
 val u00 = new Counter0(11,56)
}
showRtl(new T1)

In [ ]:
class Counter2(val end: UInt) extends ImplicitArea[UInt] {
  val willIncr = False.allowOverride
  val willClear = False.allowOverride
  def clear(): Unit = willClear := True
  def incr(): Unit = willIncr := True

  val valueNext = cloneOf(end)
  val value = RegNext(valueNext) init(0)
  val willOverflowIfInc = value === end
  val willOverflow = willOverflowIfInc && willIncr
    
  when (willOverflow){
        valueNext := 0
    }.elsewhen(willClear) {
        valueNext := 0
    }.elsewhen(willIncr) {
        valueNext := value + U(1)
    }.otherwise{
        valueNext := value  
    }
   override def implicitValue: UInt = this.value
}


In [ ]:
class T3 extends Component{
    val end  = in UInt(8 bits)
    val en = in Bool()
    val b = new Counter2(end)
    when(en)(b.incr)
    val c = out UInt()
    c := b 
    
}
showRtl(new T3)

In [ ]:
class T3 extends Component{
    val ins = in UInt(  )
    val init = in UInt(  )
    val en = in Bool()
    val outs = out UInt()
    //val t = U"8'h1A"
    //val h = History(ins,4,en,U"8'b0")
    val b = cloneOf(ins)
    b := 0
    val h = History(ins,4,en )  
    //outs := RegNext(h(2),init = U"8'd0")
    //outs := RegNext(h(2)) init( 31 )
}
 class Top extends Component{
     val ins = in UInt(8 bits)
 val t3 = new T3
     t3.ins := ins 
     t3.en := True
     
 }
showRtl(new Top)

In [ ]:
def hi(locate: String, name:String)= {
    println(s"hi, name is $name, from $locate,")
}

hi("usa","jack")
hi ("lucy", "jd")

In [ ]:
def hi( init :String)= {
    println(s"reg init  $init, ")
}
hi "323"

In [ ]:
def sum(a: Int,b: Int,c: Int) = a + b + c
val a = sum _
a(1,2,3)

In [ ]:
class SUB0 extends Component{
    val ins = in UInt() 
    val ots = out UInt()
    val tmp = Reg(cloneOf(ins)) //how to clone 
    ots := tmp
}
class Top extends Component{
    val a = in UInt(8 bits)
    val b = out UInt()
    val u = new SUB0
    u.ins :=a 
    b := u.ots
}
showRtl(new Top)

In [ ]:
var bitFlag = 0
bitFlag |=1
println(bitFlag)
//bitFlag |=2
//println(bitFlag)
bitFlag |=6

println(bitFlag)

In [ ]:
class CounterTo(val end: UInt, val inc: Bool) extends ImplicitArea[UInt] {
  val willClear = False.allowOverride

  def clear(): Unit = willClear := True

  def clearWhen(clc: Bool)={
    when(clc) {willClear := True}
    this
  }

  val valueNext = cloneOf(end)
  val value = RegNext(valueNext) init(0)
  val willOverflowIfInc = value === end
  val willOverflow = willOverflowIfInc && inc

  when (willOverflow){
    valueNext := 0
  }.elsewhen(willClear) {
    valueNext := 0
  }.elsewhen(inc) {
    valueNext := value + U(1)
  }.otherwise{
    valueNext := value
  }
  override def implicitValue: UInt = this.value
}

object CounterTo {
  def apply(end: UInt, inc: Bool ): CounterTo = new CounterTo(end, inc )
  def apply(end: UInt): CounterTo = new CounterTo(end, True)
}


In [ ]:
class Top extends Component {
    val start = in Bool()
    val end = in UInt(8 bits)
    
    //lazy val tapAddr = CounterTo(end, tapCntEn)
    //val tapCntEn = RegInit(True) setWhen(start) clearWhen(tapAddr.willOverflow)
       
    val tapCntEn = RegInit(True) 
    val tapAddr = CounterTo(end, tapCntEn)
    
    tapCntEn.setWhen(start).clearWhen(tapAddr.willOverflow)
    
}
showRtl(new Top)

## 参数协商

In [ ]:
case class FIRConfig(iqWidth: Int = 8,
                     coefWidth: Int = 8,
                     tapNumbers: Int = 33,
                     maxUpSmpTimes : Int = 5
                     ){
  def cyclesPerSmp(): Int = (tapNumbers>>1) + 1
  def tapCntMaxValue(): Int = cyclesPerSmp - 1
  val accWidth: Int = iqWidth + coefWidth + log2Up(tapNumbers)
  val fixSelWidth: Int = log2Up(coefWidth + log2Up(tapNumbers))

  private var isLast: Boolean = true
  private var next : FIRConfig = null

  def -->(that: FIRConfig): Unit = {
    isLast = false
    next = that
  }

  def interGapWidth(): Int ={
    if(isLast){
      log2Up(maxUpSmpTimes)
    } else {
      log2Up(maxUpSmpTimes) + next.interGapWidth
    }
  }
}

In [ ]:
case class FilterGroupConfig(stagesNumber: Int = 3,
                             hwFreq: HertzNumber = 200 MHz,
                             sampleFreq: HertzNumber = 1.92 MHz){
  val stagesConfig: Array[FIRConfig] = Array.fill(stagesNumber)(FIRConfig(8,8,33))
  def chain(A: FIRConfig, B: FIRConfig): FIRConfig = { A --> B ; A}
  def negotiate() ={stagesConfig.reduceRight(chain);this}
}

In [ ]:
val fgc = FilterGroupConfig()
fgc.stagesConfig(1) = FIRConfig(maxUpSmpTimes=4)
fgc.negotiate()
fgc.stagesConfig.map(x=>println(x.interGapWidth))

In [ ]:
class top extends Component{
    val a =  fgc.stagesConfig.map(x=>UInt(x.interGapWidth bits))
    val c = in Vec(a)
}
showRtl(new top)

In [ ]:
case class IQ(w: BitCount) extends Bundle{
  val I = SInt(w)
  val Q = SInt(w)

  val width = w.value

  def toBits: Bits = (I @@ Q).asBits

  def +!(that: IQ) : IQ = {
    val retWidth = (this.width max that.width)
    val ret = IQ(retWidth + 1 bits)
    ret.I := this.I + that.I.resize(retWidth + 1)
    ret.Q := this.Q + that.Q.resize(retWidth + 1)
    ret
  }

  def +(that: IQ) : IQ = {
    val ret = IQ((this.width max that.width)  bits)
    ret.I := this.I + that.I
    ret.Q := this.Q + that.Q
    ret
  }

  def *[T <: SInt](coef: T): IQ = {
    val ret = IQ(this.width + coef.getWidth bits)
    ret.I := this.I * coef
    ret.Q := this.Q * coef
    ret
  }

  def init[T <: Data](x: T): IQ = {
    this.I.initFrom(x)
    this.Q.initFrom(x)
    this
  }

  def init(x: Int): IQ = init(S(x,w))

  def clear = {
    this.I := 0
    this.Q := 0
    this
  }

  def bitSel(sel: UInt): IQ = {
    val ret = IQ(this.width bits)
    ret.I := this.I(0 until ret.I.getWidth)
    ret.Q := this.Q(0 until ret.Q.getWidth)
//    ret.I := sel.mux(0->this.I(0 until ret.I.getWidth),
//      1-> this.I(1 until ret.I.getWidth + 1),
//      2-> this.I(2 until ret.I.getWidth + 2),
//      3-> this.I(3 until ret.I.getWidth + 3),
//      4-> this.I(4 until ret.I.getWidth + 4),
//    )
    ret
  }
}

In [ ]:
case class IQ(w: BitCount) extends Bundle{
  val I = SInt(w)
  val Q = SInt(w)
  val width = w.value
  def +(that: IQ) : IQ = {
    val ret = IQ((this.width max that.width)  bits)
    ret.I := this.I + that.I 
    ret.Q := this.Q + that.Q 
    ret
  }
}

In [ ]:

class top extends Component{
    val a =  in(IQ(8 bits))
    val en = in Bool()
    
    val Acc = Reg(IQ(12 bits)) init 32  
    //Acc.noCombLoopCheck
    when(en) {
        Acc := Acc + a 
    }
}
showRtl(new top)

In [ ]:
class top extends Component{
    val a =  in(UInt(8 bits))
    val en = in Bool()
 
 val Acc = RegNextWhen(a,en) init 32
}
showRtl(new top)

In [ ]:
case class IQBD(w: Int){
    val I = SInt(w bits)
    val Q = SInt(w bits)
}

class top extends Component{
    val a =  in(IQBD(8))
    val en = in Bool()
    
    //val Acc = RegNextWhen(a,en) init 32  
 val Acc = RegNextWhen(a,en) 
}
showRtl(new top)    

In [ ]:
scala.util.Random.nextInt%256

In [ ]:
class SyncronousCheckerTesterC(v : Int) extends Component{
  val clk1, clk2, reset = in Bool()
  val input = in UInt(8 bits)
  val output = out UInt(8 bits)

  val clk1Buf = List.fill(3)(Bool).foldLeft(clk1){(i,o) => o := i; o}
  val clk2Buf = List.fill(3)(Bool).foldLeft(clk2){(i,o) => o := i; o}

  val cd1 = ClockDomain(clk1Buf, reset)
  val cd2 = ClockDomain(clk2Buf, reset)
  v match{
    case 0 =>
    case 1 => cd1.setSynchronousWith(cd2)
    case 2 => cd2.setSynchronousWith(cd1)
    case 3 => ClockDomain(clk1).setSynchronousWith(cd2)
    case 4 => ClockDomain(clk2).setSynchronousWith(cd1)
    case 5 => ClockDomain(clk1).setSynchronousWith(ClockDomain(clk2))
  }

   val logic1 = cd1 on new Area{
    val a = RegNext(input)
    val b = RegNext(a)
    output := b
  }

   val logic2 = cd2 on new Area{
    val a = out(RegNext(logic1.a))
  }
}
showRtl(new SyncronousCheckerTesterC(1))

In [ ]:
Range(1,10).foldRight(33)((i,o) => {println(s"$i -> $o");i})

## 反射

In [ ]:
case class A(x: Int) {
    val greet = "hello"
    val name = "jack"
    //prinln(nameOf(greet))
}
class B {
    val greet = "hello"
    val name = "jack"
    //prinln(nameOf(greet))
}
val a = classOf[A].getName
val b = classOf[B].getName
val a0 =new A(1)
a0.getClass.getName

In [ ]:
import scala.language.experimental.macros
import scala.reflect.macros.blackbox.Context

object Macros {

  def impl(c: Context)(x: c.Tree): c.Tree = {
    import c.universe._
    val p = x match {
      case Select(_, TermName(s)) => s
      case _ => ""
    }
    q"$p"
  }
    
  // write macros here
  def getName(x: Any): String = macro impl

}

## 获取类名

In [ ]:
object classNameOf{
  def apply(that : Any): String = that.getClass.getSimpleName.replace("$",".").split("\\.").head
}
classNameOf(A)

In [ ]:
A.getClass.getSimpleName

In [ ]:
class Thing 
object Thingy extends Thing
Thingy.getClass.getName

## 变量名

In [ ]:
import scala.annotation.tailrec
import scala.reflect.macros._
object NameOfImpl {
  def nameOf(c: Context)(expr: c.Expr[Any]): c.Expr[String] = {
    import c.universe._

    @tailrec def extract(tree: c.Tree): c.Name = tree match {
      case Ident(n) => n
      case Select(_, n) => n
      case Function(_, body) => extract(body)
      case Block(_, expr) => extract(expr)
      case Apply(func, _) => extract(func)
      case TypeApply(func, _) => extract(func)
      case _ => c.abort(c.enclosingPosition, s"Unsupported expression: $expr")
    }

    val name = extract(expr.tree).decoded
    reify {
      c.Expr[String] { Literal(Constant(name)) }.splice
    }
  }

  def nameOfType[T](c: Context)(implicit tag: c.WeakTypeTag[T]): c.Expr[String] = {
    import c.universe._
    val name = showRaw(tag.tpe.typeSymbol.name)
    reify {
      c.Expr[String] { Literal(Constant(name)) }.splice
    }
  }

  def qualifiedNameOfType[T](c: Context)(implicit tag: c.WeakTypeTag[T]): c.Expr[String] = {
    import c.universe._
    val name = showRaw(tag.tpe.typeSymbol.fullName)
    reify {
      c.Expr[String] { Literal(Constant(name)) }.splice
    }
  }
}

In [ ]:
trait NameOf {
  import scala.language.experimental.macros

  /**
    * Obtain an identifier name as a constant string.
    *
    * Example usage:
    * {{{
    *   val amount = 5
    *   nameOf(amount) => "amount"
    * }}}
    */
  def nameOf(expr: Any): String = macro NameOfImpl.nameOf

  /**
    * Obtain an identifier name as a constant string.
    *
    * This overload can be used to access an instance method without having an instance of the type.
    *
    * Example usage:
    * {{{
    *   class Person(val name: String)
    *   nameOf[Person](_.name) => "name"
    * }}}
    */
  def nameOf[T](expr: T => Any): String = macro NameOfImpl.nameOf

  /**
    * Obtain a type's unqualified name as a constant string.
    *
    * Example usage:
    * {{{
    *   nameOfType[String] => "String"
    *   nameOfType[fully.qualified.ClassName] => "ClassName"
    * }}}
    */
  def nameOfType[T]: String = macro NameOfImpl.nameOfType[T]

  /**
    * Obtain a type's qualified name as a constant string.
    *
    * Example usage:
    * {{{
    *   nameOfType[String] => "java.lang.String"
    *   nameOfType[fully.qualified.ClassName] => "fully.qualified.ClassName"
    * }}}
    */
  //def qualifiedNameOfType[T]: String = macro NameOfImpl.qualifiedNameOfType[T]
}
object NameOf extends NameOf

In [ ]:
import java.util.Date
import reflect.macros.Context
import language.experimental.macros

object CompileTime {
  def apply(): Date = macro applyImpl

  def applyImpl(c: Context)(): c.Expr[Date] = {
    import c.universe._
    val now     = System.currentTimeMillis() // this is executed during compilation!
    val nowExpr = c.Expr[Long](Literal(Constant(now)))
    val code    = reify(new Date(nowExpr.splice))
    c.Expr(code.tree)
  }
}

In [ ]:
import scala.util.Random
val a = List.fill(8)(Random.nextInt%256)

In [ ]:
import spinal.lib.soc.pinsec._
showRtl(new Pinsec(500 MHz))

In [9]:
import spinal.lib.bus.amba3.apb._
class T2 extends Component{
    val busout = master(Apb3(Apb3Config(12,32)))
    val busin = slave(Apb3(Apb3Config(12,32)))
    busout <> busin
}
showRtl(new T2)

[Runtime] SpinalHDL v1.3.6    git head : 10854057c32ae371aabc9a340c367e9bbc159fcd
[Runtime] JVM max memory : 3584.0MiB
[Runtime] Current date : 2019.10.25 16:59:12
[Progress] at 747.670 : Elaborate components
[Progress] at 747.672 : Checks and transforms
[Progress] at 747.673 : Generate Verilog
[Done] at 747.675
// Generator : SpinalHDL v1.3.6    git head : 10854057c32ae371aabc9a340c367e9bbc159fcd
// Date      : 25/10/2019, 16:59:12
// Component : T2


module T2 (
      output [11:0] busout_PADDR,
      output [0:0] busout_PSEL,
      output  busout_PENABLE,
      input   busout_PREADY,
      output  busout_PWRITE,
      output [31:0] busout_PWDATA,
      input  [31:0] busout_PRDATA,
      input   busout_PSLVERROR,
      input  [11:0] busin_PADDR,
      input  [0:0] busin_PSEL,
      input   busin_PENABLE,
      output  busin_PREADY,
      input   busin_PWRITE,
      input  [31:0] busin_PWDATA,
      output [31:0] busin_PRDATA,
      output  busin_PSLVERROR);
  assign busout_PADDR = bu

import spinal.lib.bus.amba3.apb._

defined class T2

In [13]:
import spinal.lib.bus.amba4.axi._
class T3(config: Axi4Config) extends Component{
    val busout = master(Axi4(config))
    val busin = slave(Axi4(config))
    busout <> busin
}
showRtl(new T3(Axi4Config(16,32,4)))

[Runtime] SpinalHDL v1.3.6    git head : 10854057c32ae371aabc9a340c367e9bbc159fcd
[Runtime] JVM max memory : 3584.0MiB
[Runtime] Current date : 2019.10.26 09:53:10
[Progress] at 61585.467 : Elaborate components
[Progress] at 61585.471 : Checks and transforms
[Progress] at 61585.476 : Generate Verilog
[Done] at 61585.479
// Generator : SpinalHDL v1.3.6    git head : 10854057c32ae371aabc9a340c367e9bbc159fcd
// Date      : 26/10/2019, 09:53:10
// Component : T3


module T3 (
      output  busout_aw_valid,
      input   busout_aw_ready,
      output [15:0] busout_aw_payload_addr,
      output [3:0] busout_aw_payload_id,
      output [3:0] busout_aw_payload_region,
      output [7:0] busout_aw_payload_len,
      output [2:0] busout_aw_payload_size,
      output [1:0] busout_aw_payload_burst,
      output [0:0] busout_aw_payload_lock,
      output [3:0] busout_aw_payload_cache,
      output [3:0] busout_aw_payload_qos,
      output [2:0] busout_aw_payload_prot,
      output  busout_w_valid,


import spinal.lib.bus.amba4.axi._

defined class T3